Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = '../../nlp_data_set/notmnist/notMNIST_good.pickle'

In [3]:
with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (192339, 28, 28) (192339,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (192339, 784) (192339, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, 
                                                                  logits=logits))

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Let's run this computation and iterate:

In [6]:
num_steps = 1001

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
            predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), 
                                                           valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 18.315073
Training accuracy: 7.4%
Validation accuracy: 9.7%
Loss at step 100: 2.377997
Training accuracy: 70.9%
Validation accuracy: 70.5%
Loss at step 200: 1.927313
Training accuracy: 73.9%
Validation accuracy: 72.8%
Loss at step 300: 1.682433
Training accuracy: 75.5%
Validation accuracy: 73.7%
Loss at step 400: 1.515874
Training accuracy: 76.3%
Validation accuracy: 74.2%
Loss at step 500: 1.392012
Training accuracy: 77.1%
Validation accuracy: 74.5%
Loss at step 600: 1.294382
Training accuracy: 77.7%
Validation accuracy: 74.7%
Loss at step 700: 1.214185
Training accuracy: 78.1%
Validation accuracy: 74.9%
Loss at step 800: 1.146496
Training accuracy: 78.4%
Validation accuracy: 75.0%
Loss at step 900: 1.088366
Training accuracy: 78.8%
Validation accuracy: 75.2%
Loss at step 1000: 1.037863
Training accuracy: 79.2%
Validation accuracy: 75.5%
Test accuracy: 82.2%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [7]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], 
                                        feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), 
                                                           valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 17.063629
Minibatch accuracy: 7.8%
Validation accuracy: 10.5%
Minibatch loss at step 500: 1.641600
Minibatch accuracy: 76.6%
Validation accuracy: 75.9%
Minibatch loss at step 1000: 1.274242
Minibatch accuracy: 73.4%
Validation accuracy: 76.9%
Test accuracy: 83.5%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

- 添加一个隐藏层

In [9]:
lr = 0.05
epochs = 100
batch_size =500
hidden_nodes = 1024
num_steps = 4001

In [10]:
graph = tf.Graph()
with graph.as_default():
    # datasets
    train_X = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    train_y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    valid_X = tf.constant(valid_dataset)
    test_X = tf.constant(test_dataset)
    print(train_X)

    # --------Create the first layer (1024 hidden nodes)--------
    W1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    b1 = tf.Variable(tf.zeros([hidden_nodes]))
    layer_1 = tf.nn.relu(tf.add(tf.matmul(train_X, W1), b1))
    print(layer_1)

    # --------Create second layer (10 hidden nodes)--------
    W2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    logits = tf.add(tf.matmul(layer_1, W2), b2)

    # Minimize error using cross entropy
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=train_y, logits=logits))

    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(lr).minimize(loss)

    # Predictions for the training, validation, and test data.
    pre_train_y = tf.nn.relu(tf.add(tf.matmul(train_X, W1), b1))
    train_prediction = tf.nn.softmax(tf.add(tf.matmul(pre_train_y, W2), b2))

    valid_y = tf.nn.relu(tf.add(tf.matmul(valid_X, W1), b1))
    valid_prediction = tf.nn.softmax(tf.add(tf.matmul(valid_y, W2), b2))

    test_y = tf.nn.relu(tf.add(tf.matmul(test_X, W1), b1))
    test_prediction = tf.nn.softmax(tf.add(tf.matmul(test_y, W2), b2))

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        rand_index = np.random.choice(len(train_dataset), size=batch_size, replace=False)
        batch_data = np.array([train_dataset[i] for i in rand_index])
        batch_labels = np.array([train_labels[i] for i in rand_index])
        feed_dict = {train_X : batch_data, train_y : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], 
                                        feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), 
                                                           valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Tensor("Placeholder:0", shape=(500, 784), dtype=float32)
Tensor("Relu:0", shape=(500, 1024), dtype=float32)
Initialized
Minibatch loss at step 0: 404.294312
Minibatch accuracy: 14.2%
Validation accuracy: 17.8%
Minibatch loss at step 500: 23.748795
Minibatch accuracy: 75.6%
Validation accuracy: 79.4%
Minibatch loss at step 1000: 11.906644
Minibatch accuracy: 84.8%
Validation accuracy: 80.9%
Minibatch loss at step 1500: 10.945785
Minibatch accuracy: 82.0%
Validation accuracy: 82.0%
Minibatch loss at step 2000: 8.374787
Minibatch accuracy: 85.4%
Validation accuracy: 82.5%
Minibatch loss at step 2500: 8.513839
Minibatch accuracy: 84.0%
Validation accuracy: 82.4%
Minibatch loss at step 3000: 7.214110
Minibatch accuracy: 84.6%
Validation accuracy: 82.9%
Minibatch loss at step 3500: 8.006565
Minibatch accuracy: 83.4%
Validation accuracy: 83.5%
Minibatch loss at step 4000: 7.022653
Minibatch accuracy: 83.8%
Validation accuracy: 83.4%
Test accuracy: 89.8%
